In [3]:
import sys
sys.path.insert(
    0, "/Users/anis/test_labnbook/test_3_indicateurs/traces_contribution/sources")
import numpy as np
import mysql.connector as mariadb
import pandas as pd


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

user = "root"
password = "11950022"
host = "localhost"
database_name  = "Labnbook"

try:
    conn = mariadb.connect(user=user, password=password,
                           host=host, database=database_name)
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")

# Récupération du curseur
cursor = conn.cursor()


In [ ]:
# Importer les sorties des algo qui calculent nos indicateurs  
data = pd.read_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/traces_collaboration_Anis/tmp/collab_Anis.csv")
contrib = pd.read_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/traces_contribution/tmp/Contribution.csv")['indice_collab']
data.insert(7, 'indice_contrib', contrib)
# Garder que la dérniére trace de chaque labdoc
idx_labdocs = data['id_labdoc'].diff(
)[data['id_labdoc'].diff() != 0].index.values
idx_labdocs = np.delete(idx_labdocs, 0) - 1
data = data[data.index.isin(idx_labdocs)]


data = data.drop(data[data['n_users'] == 1].index).reset_index()
data = data.drop([ "id_mission", "id_report", "id_trace","index"], axis=1)
#display(data.info(),data.head())

In [5]:
# Je récupère  l'id_action =9  (trace de modification)  de tous les labdocs dans la table trace
trace = pd.read_sql(
    " SELECT id_labdoc, action_time from trace WHERE  id_action=9 Order By id_labdoc ASC, action_time ASC", conn)


/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_35635/1125335434.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trace = pd.read_sql(


In [7]:
df2 = trace
size = df2.groupby(df2.id_labdoc,  as_index=False).size()

In [4]:
# Je fusionne trace et data avec l'intersection en id_labdoc pour avoir uniquement la trace des labdocs dont j'ai besoin 
df2 = pd.merge(data, trace, how='inner', on=['id_labdoc'])
#display(df2.head(),df2.info())

In [5]:
# Je récupére le nombre de trace modify de chaque labdoc 
size = df2.groupby(df2.id_labdoc,  as_index=False).size()
#size.head()

In [6]:
# Je rajoute ce nombre a la table 
df2 = pd.merge(df2, size, how='inner', on=['id_labdoc'])
#df2.head()

In [7]:
# Récupérer les index du changement de labdoc 
idx_time_change = df2['id_labdoc'].diff()[df2['id_labdoc'].diff() != 0].index.values

In [8]:
debut = df2.iloc[idx_time_change[:-1]].reset_index()
debut.drop(["index"],axis=1,inplace=True )
#debut

In [9]:
fin_idx = np.delete(idx_time_change - 1, 0)
fin = df2.iloc[fin_idx].reset_index()
fin.drop(["index"], axis=1, inplace=True)
#fin

In [10]:
fin["time"] = fin.action_time - debut.action_time
df = fin.drop(["action_time"], axis = 1)
df.columns = ["id_labdoc"	,"n_users",	"n_mots", "indice_collab", "indice_contrib", "n_modif", "duree"]
#df 

In [11]:
df.duree = df.duree.dt.total_seconds()
#df.info()

In [12]:
# Je recupére encore une fois la table trace 
trace2 = pd.read_sql(
    " SELECT id_labdoc, id_user from trace WHERE  id_action=9 Order By id_labdoc ASC, action_time ASC", conn)


In [13]:
# Je merge la table des traces dans la bdd avec mon data frame pour garder que les labdocs qui m'intéressent 
trace_df_merge = pd.merge(df, trace2, how='inner', on=['id_labdoc'])

In [14]:
# Je calcule l'indice de coécriture 
indic_coec = []
for _, id in enumerate(trace_df_merge["id_labdoc"].unique()):
    indic_coec.append(
        np.sum(trace_df_merge[trace_df_merge["id_labdoc"] == id]["id_user"].diff() != 0)-1)

df["indice_coec"] = indic_coec


In [15]:
# J'ai remarquer une anomalie sur certains labdoc. Il on un score de coécriture 0 et des indice de collaboration contribution non nulles. Je supprime ces labdocs 
df.drop(df[df["indice_coec"] == 0].index, axis=0, inplace=True)

#Renomer les colonnes
df.set_axis(['id_labdoc', 'n_users', 'n_mots', "collab", "equi_contrib", 'n_modif', 'duree', "pass_main"], axis=1, inplace =True)

#df.set_axis(['V', 'W', 'X', 'Y', 'Z'], axis=1, inplace=False)



In [16]:
df.to_excel(
    "/Users/anis/test_labnbook/test_3_indicateurs/comparaison/data.xlsx")
df.to_csv(
    "/Users/anis/test_labnbook/test_3_indicateurs/comparaison/data.csv")


In [17]:
df.columns

Index(['id_labdoc', 'n_users', 'n_mots', 'collab', 'equi_contrib', 'n_modif', 'duree', 'pass_main'], dtype='object')

In [10]:
trace = pd.read_sql(
    " SELECT id_labdoc,id_user, action_time from trace WHERE  id_action=9 Order By id_labdoc ASC, action_time ASC", conn)

/var/folders/qz/t1vzp79913l2sfbf0b3d9sdr0000gn/T/ipykernel_35635/899833016.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  trace = pd.read_sql(


In [11]:
trace

,id_labdoc,id_user,action_time
0,311,2,2018-12-06 16:48:31
1,311,2,2018-12-06 16:48:51
2,311,2,2018-12-06 16:49:11
3,311,2,2018-12-06 16:50:11
4,311,2,2018-12-06 16:50:31
...,...,...,...
4868733,491781,11801,2021-09-08 20:08:04
4868734,491781,11801,2021-09-08 20:08:34
4868735,491781,11801,2021-09-08 20:09:04
4868736,491781,11801,2021-09-08 20:09:34


In [19]:
df = trace[trace["id_labdoc"] == 453634]
df


,id_labdoc,id_user,action_time
4477710,453634,10460,2021-03-29 08:28:15
4477711,453634,10460,2021-03-29 08:28:45
4477712,453634,10460,2021-03-29 08:29:15
4477713,453634,10460,2021-03-29 08:29:45
4477714,453634,10460,2021-03-29 08:30:15
4477715,453634,10460,2021-03-29 08:30:45
4477716,453634,10460,2021-03-29 08:31:15
4477717,453634,10460,2021-03-29 08:31:45
4477718,453634,10460,2021-03-29 08:32:15
4477719,453634,10460,2021-03-29 08:32:45


In [23]:
df.groupby(df.id_user,  as_index=False).size()

,id_user,size
0,10446,29
1,10449,89
2,10455,13
3,10460,34
